In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn import metrics
from sklearn.linear_model import SGDClassifier
from scipy import sparse as sp
import matplotlib.pyplot as plt
import pickle



SEED = 1337

df = pd.read_csv('ner_short.csv', index_col=0)
df.head()


,next-next-pos,next-next-word,next-pos,next-word,pos,prev-pos,prev-prev-pos,prev-prev-word,prev-word,sentence_idx,word,tag
0,NNS,demonstrators,IN,of,NNS,__START1__,__START2__,__START2__,__START1__,1.0,Thousands,O
1,VBP,have,NNS,demonstrators,IN,NNS,__START1__,__START1__,Thousands,1.0,of,O
2,VBN,marched,VBP,have,NNS,IN,NNS,Thousands,of,1.0,demonstrators,O
3,IN,through,VBN,marched,VBP,NNS,IN,of,demonstrators,1.0,have,O
4,NNP,London,IN,through,VBN,VBP,NNS,demonstrators,have,1.0,marched,O


In [2]:
tdf = df.set_index('sentence_idx')
tdf['length'] = df.groupby('sentence_idx').tag.count()
df = tdf.reset_index(drop=False)
le = LabelEncoder()
df['pos'] = le.fit_transform(df.pos)
df['next-pos'] = le.fit_transform(df['next-pos'])
df['next-next-pos'] = le.fit_transform(df['next-next-pos'])
df['prev-pos'] = le.fit_transform(df['prev-pos'])
df['prev-prev-pos'] = le.fit_transform(df['prev-prev-pos'])

y = LabelEncoder().fit_transform(df.tag)

df_train, df_test, y_train, y_test = train_test_split(df, y, stratify=y, 
                                                                      test_size=0.25, random_state=SEED, shuffle=True)
print('train', df_train.shape[0])
print('test', df_test.shape[0])

train 50155
test 16719


In [3]:
from gensim.models.word2vec import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import TransformerMixin
from collections import defaultdict

   
class Word2VecWrapper(TransformerMixin):
    def __init__(self, window=5,negative=5, size=100, iter=100, is_cbow=False, random_state=SEED):
        self.window_ = window
        self.negative_ = negative
        self.size_ = size
        self.iter_ = iter
        self.is_cbow_ = is_cbow
        self.w2v = None
        self.random_state = random_state
        
    def get_size(self):
        return self.size_

    def fit(self, X, y=None):
        """
        X: list of strings
        """
        sentences_list = [x.split() for x in X]
        self.w2v = Word2Vec(sentences_list, 
                            window=self.window_,
                            negative=self.negative_, 
                            size=self.size_, 
                            iter=self.iter_,
                            sg=not self.is_cbow_, seed=self.random_state)

        return self
    
    def has(self, word):
        return word in self.w2v

    def transform(self, X):
        """
        X: a list of words
        """
        if self.w2v is None:
            raise Exception('model not fitted')
        return np.array([self.w2v[w] if w in self.w2v else np.zeros(self.size_) for w in X ])
    

In [4]:
%%time
# we have to fit embedding models on whole dataset as they depend on word order
from gensim.models.word2vec import Word2Vec


# notice, that our dataset has window=2
sentences_list = [x.strip() for x in ' '.join(df.word).split('.')]

w2v_cbow = Word2VecWrapper(window=2, negative=5, size=300, iter=300, is_cbow=True, random_state=SEED)
w2v_cbow.fit(sentences_list)

CPU times: user 49.7 s, sys: 152 ms, total: 49.8 s
Wall time: 16.8 s


In [6]:
embeding = w2v_cbow
encoder_pos = OneHotEncoder()
X_train = sp.hstack([
    embeding.transform(df_train.word),
    embeding.transform(df_train['next-word']),
    embeding.transform(df_train['next-next-word']),
    embeding.transform(df_train['prev-word']),
    embeding.transform(df_train['prev-prev-word']),
    encoder_pos.fit_transform(df_train[['pos','next-pos','next-next-pos','prev-pos','prev-prev-pos']])
])
X_test = sp.hstack([
    embeding.transform(df_test.word),
    embeding.transform(df_test['next-word']),
    embeding.transform(df_test['next-next-word']),
    embeding.transform(df_test['prev-word']),
    embeding.transform(df_test['prev-prev-word']),
    encoder_pos.transform(df_test[['pos','next-pos','next-next-pos','prev-pos','prev-prev-pos']])
])

model2 = GridSearchCV(SGDClassifier(verbose=2, random_state=SEED), {"loss": ["hinge","log","modified_huber"],
                                                 "learning_rate":["optimal", "invscaling"],
                                                                    "power_t":[0.1, 1],
                                                                    "eta0":[1, 10]})

model2.fit(X_train, y_train)
with open("model2.pkl", "wb") as io:
    pickle.dump(model2, io)
print(model2.best_params_)
print('train', metrics.f1_score(y_train, model2.predict(X_train), average='macro'))
print('test', metrics.f1_score(y_test, model2.predict(X_test), average='macro'))

/usr/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


-- Epoch 1
Norm: 71.44, NNZs: 1627, Bias: -5.977479, T: 33433, Avg. loss: 1.026961
Total training time: 0.07 seconds.
-- Epoch 2
Norm: 41.96, NNZs: 1632, Bias: -6.009357, T: 66866, Avg. loss: 0.058370
Total training time: 0.15 seconds.
-- Epoch 3


Norm: 30.51, NNZs: 1632, Bias: -6.023543, T: 100299, Avg. loss: 0.028737
Total training time: 0.22 seconds.
-- Epoch 4
Norm: 24.65, NNZs: 1635, Bias: -6.028373, T: 133732, Avg. loss: 0.018201
Total training time: 0.30 seconds.
-- Epoch 5
Norm: 21.29, NNZs: 1639, Bias: -6.032128, T: 167165, Avg. loss: 0.013016
Total training time: 0.37 seconds.
-- Epoch 1


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


Norm: 67.20, NNZs: 1621, Bias: -5.819603, T: 33433, Avg. loss: 0.808804
Total training time: 0.07 seconds.
-- Epoch 2
Norm: 37.38, NNZs: 1626, Bias: -5.843602, T: 66866, Avg. loss: 0.016632
Total training time: 0.15 seconds.
-- Epoch 3
Norm: 26.09, NNZs: 1627, Bias: -5.848516, T: 100299, Avg. loss: 0.005012
Total training time: 0.22 seconds.
-- Epoch 4


...

Norm: 55.67, NNZs: 1706, Bias: 4.915794, T: 200620, Avg. loss: 0.393380
Total training time: 0.90 seconds.
-- Epoch 5


Norm: 47.05, NNZs: 1706, Bias: 4.915640, T: 250775, Avg. loss: 0.300313
Total training time: 1.11 seconds.
{'eta0': 1, 'learning_rate': 'optimal', 'loss': 'log', 'power_t': 0.1}


[Parallel(n_jobs=1)]: Done  17 out of  17 | elapsed:   19.3s finished


train 0.749402043404


test 0.620379295009
